 ### 1. Pacotes

In [422]:
import pickle
import nltk
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
import unidecode
import gensim
from gensim import corpora
import pandas as pd

In [423]:
# nltk.download()

### 2. Lendo dicionário de ementas
##### Obs: Criei este dicionário através de várias raspagens (ver arquivo "baixar_leis").

In [424]:
with open('dict_ementas.pickle', 'rb') as handle:
    dict_ementas = pickle.load(handle)

### 3. Limpando dados das ementas
##### Excluindo números, stopwords etc.

#### 3.1. Criando função para limpeza (substituição de números e termos inúteis, do tipo "lei" ou "artigo", e eliminação de stopwords)

In [425]:
stop = set(stopwords.words('portuguese'))

def limpa_texto(string_):
    string_limpa = string_.lower()
    string_limpa = unidecode.unidecode(string_limpa)
    string_limpa = string_limpa.replace("."," ").replace(","," ").replace("-"," ").replace(":"," ").replace(";"," ").replace("  "," ").replace(" lei "," ").replace(" decreto "," ").replace(" paragrafo "," ").replace(" janeiro "," ").replace(" fevereiro "," ").replace(" março "," ").replace(" marco "," ").replace(" abril "," ").replace(" maio "," ").replace(" junho "," ").replace(" julho "," ").replace(" agosto "," ").replace(" setembro "," ").replace(" outubro "," ").replace(" novembro "," ").replace(" dezembro "," ").replace("\""," ").replace("\""," ").replace("'"," ").replace("nº"," ").replace("ª"," ").replace("r$"," ").replace("$"," ").replace("/xa0"," ").replace(" altera "," ").replace("altera "," ").replace("autoriza "," ").replace(" n "," ").replace("  "," ").replace(" estado "," ").replace(" providencias "," ").replace(" outras "," ").replace(" dispoe "," ").replace(" sobre "," ").replace("santa catarina"," ").replace("pernambuco"," ").replace(" acre "," ").replace("alagoas"," ").replace("amazonas"," ").replace(" amapa "," ").replace("bahia"," ").replace("sergipe"," ").replace("paraiba"," ").replace("rio grande do norte"," ").replace("sergipe"," ").replace("piaui"," ").replace("maranhao"," ").replace("ceara"," ").replace("tocantins"," ").replace("distrito federal"," ").replace("goias"," ").replace("minas gerais"," ").replace("sao paulo"," ").replace("rio de janeiro"," ").replace("espirito santo"," ").replace("rio grande do sul"," ").replace("parana"," ").replace("mato grosso do sul"," ").replace("mato grosso"," ").replace(" para "," ").replace("roraima"," ").replace("rondonia"," ")
    string_limpa = ''.join([i for i in string_limpa if not i.isdigit()])
    string_limpa = " ".join([i for i in string_limpa.split() if i not in stop])
    string_limpa = ''.join(ch for ch in string_limpa if ch not in set(string.punctuation))
    string_limpa = " ".join(lemma.lemmatize(word) for word in string_limpa.split())
    return string_limpa

#### 3.2. Criando lista com todas as ementas, executando limpeza e tirando termos de 2 ou menos caracteres

In [426]:
doc_complete = []
for i in dict_ementas.values():
    for j in i:
        doc_complete.append(j)

doc_clean = [limpa_texto(doc).split() for doc in doc_complete]
doc_clean = [[token for token in doc if len(token) > 2] for doc in doc_clean]

In [427]:
print(dict_ementas["AL"][0:11])
print(doc_complete[0:11])
print(doc_clean[0:11])

['ALTERA A LEI ESTADUAL Nº 5.077, DE 12 DE JUNHO DE 1989, QUE INSTITUI O CÓDIGO TRIBUTARIO DO ESTADO DE ALAGOAS, RELATIVAMENTE AO IMPOSTO SOBRE TRANSMISSÃO CAUSA MORTIS E DOAÇÃO DE QUAISQUER BENS OU DIREITOS - ITCD.\xa0', 'ALTERA A LEI ESTADUAL N° 6.555, DE 30 DE DEZEMBRO DE 2004, QUE DISPÕE SOBRE O TRATAMENTO TRIBUTÁRIO RELATIVO AO IMPOSTO SOBRE A PROPRIEDADE DE VEÍCULOS AUTOMOTORES - IPVA.\xa0', 'ALTERA A LEI ESTADUAL Nº 5.077, DE 12 DE JUNHO DE 1989, QUE DISPÕE SOBRE NORMAS RELATIVAS AOS TRIBUTOS DE COMPETÊNCIA DO ESTADO.\xa0', 'ALTERA A LEI ESTADUAL Nº7.070, DE 15 DE JULHO DE 2009, QUE AUTORIZA O PODER EXECUTIVO A CONTRATAR E GARANTIR FINANCIAMENTO JUNTO AO BANCO NACIONAL DE DESENVOLVIMENTO ECONOMICO E SOCIAL - BNDES, NO ÂMBITO DO PROGRAMA DE MODERNIZAÇÃO DA ADMINISTRAÇÃO DAS RECEITAS E DA GESTÃO FISCAL, FINANCEIRA E PATRIMONIAL DAS ADMINISTRAÇÕES ESTADUAIS- PMAE\xa0', 'DISPÕE SOBRE O RATEIO DAS SOBRAS DE RECURSOS DO FUNDO DE MANUTENÇÃO E DESENVOLVIMENTO DA EDUCAÇÃO BÁSICA E DE VAL

### 4. Analise

#### 4.1. Gerando bigramas

In [409]:
## Nao utilizei este bloco na versao final, pois o modelo sem bigramas se mostrou bem mais coerente.
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(doc_clean, min_count=20)
for idx in range(len(doc_clean)):
    for token in bigram[doc_clean[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            doc_clean[idx].append(token)

/Applications/anaconda/lib/python3.6/site-packages/gensim/models/phrases.py:316: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


#### 4.2. Pesquisando os tópicos

##### 4.2.1. Matriz DTM

In [429]:
dictionary = corpora.Dictionary(doc_clean)
dictionary.filter_extremes(no_below=20, no_above=0.9)

In [430]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [431]:
doc_term_matrix

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)],
 [(0, 1), (3, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1)],
 [(0, 1), (12, 1), (13, 1)],
 [(0, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1)],
 [(20, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 3),
  (34, 2),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1)],
 [(12, 1), (25, 1), (39, 1), (40, 1), (41, 1), (42, 1)],
 [(0, 1), (1, 1), (2, 1), (7, 1)],
 [(29, 1), (43, 1), (44, 1)],
 [(0, 1), (45, 1), (46, 1), (47, 1), (48, 1)],
 [(22, 1), (45, 1), (46, 1), (47, 1), (49, 1), (50, 1), (51, 1), (52, 1)],
 [(31, 1),
  (53, 2),
  (54, 1),
  (55, 1),
  (56, 2),
  (57, 2),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1)],
 [(14, 1),
  (15, 1),
  (56, 1),
  (57, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1)],
 [(69, 1), (70, 1), (71, 1), (72, 1)],
 [(0, 1)],
 [(0

##### 4.2.2. Modelo LDA

In [432]:
Lda = gensim.models.ldamodel.LdaModel

In [433]:
#ldamodel = Lda(doc_term_matrix, num_topics=7, id2word = dictionary, passes=100)
ldamodel_2 = Lda(doc_term_matrix, num_topics=7, id2word = dictionary, passes=100)

In [449]:
for i in ldamodel.print_topics(num_topics=7, num_words=10):
    print(i)

(0, '0.094*"especifica" + 0.079*"titulo" + 0.068*"concede" + 0.047*"denominacao" + 0.046*"providencias" + 0.040*"cidadao" + 0.029*"lei" + 0.024*"estadual" + 0.022*"estabelece" + 0.022*"indica"')
(1, '0.049*"institui" + 0.040*"estadual" + 0.035*"dispoe" + 0.030*"programa" + 0.023*"ambito" + 0.021*"educacao" + 0.021*"saude" + 0.021*"ensino" + 0.020*"acrescenta" + 0.020*"dispositivos"')
(2, '0.146*"dispoe" + 0.048*"obrigatoriedade" + 0.044*"publicos" + 0.042*"servicos" + 0.035*"rio" + 0.034*"ambito" + 0.030*"publica" + 0.025*"seguranca" + 0.025*"concessao" + 0.023*"veiculos"')
(3, '0.179*"publica" + 0.167*"utilidade" + 0.130*"declara" + 0.088*"associacao" + 0.080*"sede" + 0.076*"municipio" + 0.038*"foro" + 0.027*"entidade" + 0.016*"reconhece" + 0.016*"especifica"')
(4, '0.146*"municipio" + 0.053*"cultural" + 0.046*"denomina" + 0.040*"estadual" + 0.040*"sao" + 0.032*"rodovia" + 0.030*"escola" + 0.027*"localizada" + 0.025*"patrimonio" + 0.022*"doar"')
(5, '0.081*"poder" + 0.073*"executivo" 

#### 5. Atribuindo tópicos para os documentos originais

In [438]:
doclda = ldamodel[doc_term_matrix]

In [439]:
print(list(dict_ementas.values())[0][1])
print(doc_complete[1])
print(doc_clean[1])
print(doclda[1])

ALTERA A LEI ESTADUAL N° 6.555, DE 30 DE DEZEMBRO DE 2004, QUE DISPÕE SOBRE O TRATAMENTO TRIBUTÁRIO RELATIVO AO IMPOSTO SOBRE A PROPRIEDADE DE VEÍCULOS AUTOMOTORES - IPVA. 
ALTERA A LEI ESTADUAL N° 6.555, DE 30 DE DEZEMBRO DE 2004, QUE DISPÕE SOBRE O TRATAMENTO TRIBUTÁRIO RELATIVO AO IMPOSTO SOBRE A PROPRIEDADE DE VEÍCULOS AUTOMOTORES - IPVA. 
['estadual', 'ndeg', 'tratamento', 'tributario', 'relativo', 'imposto', 'propriedade', 'veiculos', 'automotores', 'ipva']
[(0, 0.017862638720854387), (1, 0.017885032489658986), (2, 0.51784297419666514), (3, 0.017870553068553521), (4, 0.017866397241972858), (5, 0.0178619786249898), (6, 0.39281042565730523)]


In [440]:
dict_ementas

{'AC': ['Institui o Sistema Integrado de Habitação do Estado do Acre – SIHAB e dispõe sobre a sua gestão e manutenção.lei3213',
  'Concede o Título de Cidadão Acreano ao Sr. Marcelo Grando.3-188',
  'Altera a Lei n. 2.005, de 9 de junho de 2008 que “Cria o Sistema Integrado de Segurança Pública e autoriza a instituição de suas regionais”.lei3210',
  'Reconhece o Município de Xapuri como Centro Cultural e Ecológico do Estado.3.133',
  'Altera a Lei n. 3.153, de 29 de julho de 2016, que obriga os fornecedores de bens e serviços, que atuam no Estado, a fixar data e turno para entrega dos produtos ou realização dos serviços aos consumidores.\xa0Lei 3.221',
  'Institui o Fundo de Equilíbrio Fiscal do Estado do Acre – FEFAC e condiciona a fruição de incentivos ou benefícios fiscais ou financeiros à efetivação de depósitos no referido fundo, nos termos do Convênio ICMS 42, de 3 de maio de 2016.lei3216',
  '“Autoriza a concessão de uso dos silos graneleiros situados nos Municípios de Rio Branc

#### Criando dataframe com ementas classificadas

In [441]:
df_ementas = pd.DataFrame()
#columns=["estado", "ementa", "ementa_limpa", "topicos_prob"]

In [442]:
lista_estado = []
lista_ementa = []

for i in dict_ementas.keys():
    for j in dict_ementas[i]:
        lista_estado.append(i)
        lista_ementa.append(j)

In [443]:
df_ementas["estado"] = lista_estado
df_ementas["ementa"] = lista_ementa
df_ementas["ementa_limpa"] = doc_clean
df_ementas["prob_topicos"] = doclda

In [444]:
def coluna_maior_topico(x):
    top_max = 0
    val_top_max = 0
    for i in x:
        if i[1] > val_top_max:
            top_max = i[0]
            val_top_max = i[1]
    return top_max

def coluna_maior_topico_valor(x):
    top_max = 0
    val_top_max = 0
    for i in x:
        if i[1] > val_top_max:
            top_max = i[0]
            val_top_max = i[1]
    return val_top_max

def resumo_maior_topico(x):
    for i in ldamodel.print_topics(num_topics=7, num_words=10):
        if i[0] == x:
            resumo = i[1]
    return resumo

In [445]:
df_ementas["maior_topico"] = df_ementas["prob_topicos"].map(coluna_maior_topico)
df_ementas["maior_topico_valor"] = df_ementas["prob_topicos"].map(coluna_maior_topico_valor)
df_ementas["resumo_maior_topico"] = df_ementas["maior_topico"].map(resumo_maior_topico)